# Section 4 - Container images, where to find them and how to build them
## 35 - What's an image and what is it not
An image is a set of application binaries, their dependancies as well as the metadata on the image and how it should run. It doesn't contain an OS, the host on which the application runs will provide the kernel/OS/Drives


## 36- The mighty hub: using docker hub registry images 

The image registry can be found at hub.docker.com

Each application can have multiple images on the registry (e.g: nginx has ~90,000 images). To begin with, an easy choice is to start with the official images that are curated by the team at Docker (who usualy work hand in hand with the team that created the software).

You can find documentation on how to run the image within its page on the hub.

There are multiple versions of most official images. Each version can have more than one tag.

## 37- Images and their layers: discover the image cache

Images don't just come as a single piece of data. They're usualy stacks of code that build on top of each other (much like commits). This can be seen when downloading new images and there 

They are designed using the union file concept of making layers of changes.

In [1]:
%%bash
docker image history nginx:latest

IMAGE          CREATED        CREATED BY                                      SIZE      COMMENT
f8f4ffc8092c   2 months ago   /bin/sh -c #(nop)  CMD ["nginx" "-g" "daemon…   0B        
<missing>      2 months ago   /bin/sh -c #(nop)  STOPSIGNAL SIGQUIT           0B        
<missing>      2 months ago   /bin/sh -c #(nop)  EXPOSE 80                    0B        
<missing>      2 months ago   /bin/sh -c #(nop)  ENTRYPOINT ["/docker-entr…   0B        
<missing>      2 months ago   /bin/sh -c #(nop) COPY file:09a214a3e07c919a…   4.61kB    
<missing>      2 months ago   /bin/sh -c #(nop) COPY file:0fd5fca330dcd6a7…   1.04kB    
<missing>      2 months ago   /bin/sh -c #(nop) COPY file:0b866ff3fc1ef5b0…   1.96kB    
<missing>      2 months ago   /bin/sh -c #(nop) COPY file:65504f71f5855ca0…   1.2kB     
<missing>      2 months ago   /bin/sh -c set -x     && addgroup --system -…   64MB      
<missing>      2 months ago   /bin/sh -c #(nop)  ENV PKG_RELEASE=1~buster     0B        
<missing>     

The above is a history of the image layers. Every image start with a blank base layer called "scratch", every change since then on the image file system is an added layer. Every new layer has its own unique hash code (sha265).

Images being stacks of layers can "share" cached layers that are common across images (e.g: Two images can use the same scratch layer of Debian Jessie)

In deploying containers, if you modify one of the files in the base layer that is shared among containers. Only the difference is copied out. This is known as copy on write, the differencing elements in the layers are copied into the container that does the changes (not affecting the other containers).

The docker inspect command return metadata on the image in JSON format.

In [2]:
%%bash
docker inspect nginx

[
    {
        "Id": "88fdc087fc760fde3a8f4246b962d44e4f20db776212e0646ead4b37e721dc57",
        "Created": "2021-10-07T10:42:16.48658428Z",
        "Path": "/docker-entrypoint.sh",
        "Args": [
            "nginx",
            "-g",
            "daemon off;"
        ],
        "State": {
            "Status": "exited",
            "Running": false,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 0,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2021-10-12T15:29:01.758865157Z",
            "FinishedAt": "2021-10-12T15:52:30.706158242Z"
        },
        "Image": "sha256:f8f4ffc8092c956ddd6a3a64814f36882798065799b8aedeebedf2855af3395b",
        "ResolvConfPath": "/var/lib/docker/containers/88fdc087fc760fde3a8f4246b962d44e4f20db776212e0646ead4b37e721dc57/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/88fdc087fc760fde3a8f4246b962d44e4f20db776

## 38- Image taggin and pushing to Docker hub

Image tag syntax: \<user>/\<repo>:\<tag>

A tag is a pointer to a specific image commit (IMAGE ID). It a many to one relation between TAG and IMAGE ID (N:1)

In [4]:
%%bash

docker image tag --help


Usage:  docker image tag SOURCE_IMAGE[:TAG] TARGET_IMAGE[:TAG]

Create a tag TARGET_IMAGE that refers to SOURCE_IMAGE


You can retag new tags to existing images

In [5]:
%%bash
docker image tag nginx karimitn/nginx

In [7]:
%%bash
#you can see the new tag for karimitn/nginx having the same IMAGE ID as the original nginx image
docker image ls 

REPOSITORY       TAG       IMAGE ID       CREATED        SIZE
httpd            latest    bc9d3c5a7455   2 months ago   138MB
ubuntu           latest    597ce1600cf4   2 months ago   72.8MB
karimitn/nginx   latest    f8f4ffc8092c   2 months ago   133MB
nginx            latest    f8f4ffc8092c   2 months ago   133MB
mysql            latest    2fe463762680   2 months ago   514MB
alpine           latest    14119a10abf4   3 months ago   5.6MB


In [8]:
%%bash
#The new tag won't show in Docker Hub untill you run a push command
docker image push karimitn/nginx

Using default tag: latest
The push refers to repository [docker.io/karimitn/nginx]
65e1ea1dc98c: Preparing
88891187bdd7: Preparing
6e109f6c2f99: Preparing
0772cb25d5ca: Preparing
525950111558: Preparing
476baebdfbf7: Preparing
476baebdfbf7: Waiting
65e1ea1dc98c: Mounted from library/nginx
0772cb25d5ca: Mounted from library/nginx
6e109f6c2f99: Mounted from library/nginx
525950111558: Mounted from library/nginx
88891187bdd7: Mounted from library/nginx
476baebdfbf7: Mounted from library/httpd
latest: digest: sha256:39065444eb1acb2cfdea6373ca620c921e702b0f447641af5d0e0ea1e48e5e04 size: 1570


## 39- Building Images: The Dockerfile Basics
Note: Use the dockerfile-sample-1 folder for this section

The default name for such files is Dockerfile but any other name can be supplied. You can use the flag -f to specify which file you want to use.

Example:\
docker build -f some-dockerfile

Each stanza in the Dockerfile is an actual layer and thus the order in which they are run matters a lot.

### Structure: 

- FROM command is required, it tells docker which image layer this is building up from
- ENV is used to set environemnt variables
- RUN is used to run shell command during the build process (usualy installing packages, zipping, file edits,etc)
- EXPOSE set which ports are exposed (by default no ports are exposed in a container to a virtual network)
- CMD is the final command that is added (it is used every time a new container is launched from the image or restart a stopped container)




In [17]:
%%bash
pwd
cd ./udemy-docker-mastery/dockerfile-sample-1/
pwd
ls -ltra
cat Dockerfile


/Users/karim/docker_course/Docker_Mastery
/Users/karim/docker_course/Docker_Mastery/udemy-docker-mastery/dockerfile-sample-1
total 8
drwxr-xr-x   3 karim  staff    96 Oct  7 13:08 .
-rw-r--r--   1 karim  staff  2471 Oct  7 13:08 Dockerfile
drwxr-xr-x  37 karim  staff  1184 Oct  7 13:08 ..
# NOTE: this example is taken from the default Dockerfile for the official nginx Docker Hub Repo
# https://hub.docker.com/_/nginx/
# NOTE: This file is slightly different than the video, because nginx versions have been updated 
#       to match the latest standards from docker hub... but it's doing the same thing as the video
#       describes
FROM debian:stretch-slim
# all images must have a FROM
# usually from a minimal Linux distribution like debian or (even better) alpine
# if you truly want to start with an empty container, use FROM scratch

ENV NGINX_VERSION 1.13.6-1~stretch
ENV NJS_VERSION   1.13.6.0.1.14-1~stretch
# optional environment variable that's used in later lines and set as envvar wh

## 40- Building Images: Running Docker Buils

When you run the above Dockerfile it will pull the Debian:Jessie image from Docker Hub into the cache then it will execute the subsequent commands inside the Docker Engine and cache each of those layers

In [19]:
%%bash
cd ./udemy-docker-mastery/dockerfile-sample-1/
docker image build -t customnginx .

#1 [internal] load build definition from Dockerfile
#1 sha256:fbcc3e1dfd16153e1791aaea187dce3c14044683fffc3e50a656dff4948d298b
#1 transferring dockerfile: 2.52kB 0.0s done
#1 DONE 0.1s

#2 [internal] load .dockerignore
#2 sha256:97be39dc2318b56ddbb90d560a4b242787b8a73edfca8716a0b1c3147eda2af9
#2 transferring context: 2B done
#2 DONE 0.1s

#3 [internal] load metadata for docker.io/library/debian:stretch-slim
#3 sha256:4be36a932feb4d7c74e2e515db686e30225b46e466053f9479f5d76198ea2fbc
#3 ...

#4 [auth] library/debian:pull token for registry-1.docker.io
#4 sha256:da59642aac4d40b680773033d77566fb4560fcfdad3107d0fc6f25ca01449ba1
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/debian:stretch-slim
#3 sha256:4be36a932feb4d7c74e2e515db686e30225b46e466053f9479f5d76198ea2fbc
#3 DONE 6.7s

#7 [1/3] FROM docker.io/library/debian:stretch-slim@sha256:e4d331c0d56213e3d1fdf72b29401edb04805380c4029fc776e74d89264f6966
#7 sha256:328d4778f3888c9c72ed5bb10083b71b43708087d2166bad04dd7e1bc3bb7d9